In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

#trying sentence transformers

In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.0225026  -0.0782918  -0.02303076 ... -0.00827925  0.02652694
  -0.00201898]
 [ 0.04170239  0.00109739 -0.01553419 ... -0.02181625 -0.06359357
  -0.00875283]]


using hugging face end point, getting just the direct embedding and not passing it to cosine similarity function yet, for checking for semantics


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 0.0225, -0.0783, -0.0230,  ..., -0.0083,  0.0265, -0.0020],
        [ 0.0417,  0.0011, -0.0155,  ..., -0.0218, -0.0636, -0.0088]])


trying sentence transformer with same sentences and then checking if cosine similarity as 1.

In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'This is an example sentence']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# Calculate cosine similarity between the two sentence embeddings
cosine_similarity = F.cosine_similarity(sentence_embeddings[0], sentence_embeddings[1], dim=0)

print("Cosine similarity score between the sentences:")
print(cosine_similarity.item())


Cosine similarity score between the sentences:
1.0000001192092896


#trying out levenshtein


In [9]:
import Levenshtein

# Sentences we want to compare
sentence1 = 'This is an example sentence'
sentence2 = 'This is an example sentence'

# Compute Levenshtein distance
lev_distance = Levenshtein.distance(sentence1, sentence2)

# Normalize the distance to get a similarity score (0 to 1)
max_len = max(len(sentence1), len(sentence2))
lev_similarity = 1 - (lev_distance / max_len)

print("Levenshtein similarity score between the sentences:")
print(lev_similarity)


Levenshtein similarity score between the sentences:
1.0


#sentence transformer wil 2-3 rows from csv file


In [12]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Function to read data from CSV and concatenate all columns into a single string per row
def read_csv_and_concatenate(file_path):
    df = pd.read_csv(file_path)
    concatenated_sentences = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1).tolist()
    return concatenated_sentences

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Function to compute cosine similarity between two sentences
def compute_cosine_similarity(sentences):
    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # Calculate cosine similarity between the two sentence embeddings
    cosine_similarity = F.cosine_similarity(sentence_embeddings[0], sentence_embeddings[1], dim=0)

    return cosine_similarity.item()

# Main code
if __name__ == "__main__":
    # Path to the CSV file
    csv_file_path = 'test2.csv'  # Change this to your actual CSV file path

    # Read data from CSV and concatenate columns
    sentences = read_csv_and_concatenate(csv_file_path)


    # Compute cosine similarity between the two concatenated rows
    similarity_score = compute_cosine_similarity(sentences)
    print("Cosine similarity score between the rows:")
    print(similarity_score)



Cosine similarity score between the rows:
0.5362727046012878


using array for quick testing instead of csv

In [13]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Function to concatenate all values in each dictionary into a single string
def concatenate_values(data):
    concatenated_sentences = [' '.join([str(item['name']), str(item['about']), str(item['age'])]) for item in data]
    return concatenated_sentences

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Function to compute cosine similarity between two sentences
def compute_cosine_similarity(sentences):
    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # Calculate cosine similarity between the two sentence embeddings
    cosine_similarity = F.cosine_similarity(sentence_embeddings[0], sentence_embeddings[1], dim=0)

    return cosine_similarity.item()

# Main code
if __name__ == "__main__":
    # Array of dictionaries
    data = [
        {'name': 'Alice', 'about': 'Data Scientist', 'age': 29},
        {'name': 'Bob', 'about': 'Software Engineer', 'age': 35}
    ]

    # Concatenate values
    sentences = concatenate_values(data)

    # Compute cosine similarity between the two concatenated rows
    similarity_score = compute_cosine_similarity(sentences)
    print("Cosine similarity score between the rows:")
    print(similarity_score)


Cosine similarity score between the rows:
0.6020846962928772


#using both levenshtein and transformer and then an average score for both.


trying out multiple target profiles in array format and checking similarity


In [30]:
import Levenshtein
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Function to concatenate profile values into a single string
def concatenate_profile(profile):
    return f"{profile['name']} {profile['about']} {profile['age']}"

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Function to compute cosine similarity between a source profile and a list of target profiles
def compute_similarity_scores(source_profile, target_profiles):
    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

    # Concatenate profile values
    source_sentence = concatenate_profile(source_profile)
    target_sentences = [concatenate_profile(profile) for profile in target_profiles]

    # Tokenize sentences
    encoded_input = tokenizer([source_sentence] + target_sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # Calculate cosine similarities
    source_embedding = sentence_embeddings[0]
    target_embeddings = sentence_embeddings[1:]
    cosine_similarities = F.cosine_similarity(source_embedding.unsqueeze(0), target_embeddings, dim=1)

    # Calculate Levenshtein similarities
    levenshtein_similarities = [Levenshtein.ratio(source_sentence, target_sentence) for target_sentence in target_sentences]

    # Combine both similarities
    combined_similarities = [(cos_sim*0.85 + lev_sim*0.15) for cos_sim, lev_sim in zip(cosine_similarities.tolist(), levenshtein_similarities)]

    return combined_similarities

# Main code
if __name__ == "__main__":
    # Source profile
    source_profile = {'name': 'Alice2345', 'about': 'works as a Data Scientist at XYZ', 'age': 29}

    # List of target profiles
    target_profiles = [
        {'name': 'Alice7890', 'about': 'Data Engineer', 'age': 35},
        {'name': 'Alice3890', 'about': 'Data Scientist, started working at XYZ', 'age': 29},
        {'name': 'Alice_9073', 'about': 'Hi, I started working as a Data Scientist at XYZ', 'age': 28}
    ]

    # Compute similarity scores between the source profile and the target profiles
    similarity_scores = compute_similarity_scores(source_profile, target_profiles)

    # Print similarity scores with corresponding profiles
    for score, profile in zip(similarity_scores, target_profiles):
        print(f"Combined similarity score with '{profile}': {score}")


Combined similarity score with '{'name': 'Alice7890', 'about': 'Data Engineer', 'age': 35}': 0.5700383264414022
Combined similarity score with '{'name': 'Alice3890', 'about': 'Data Scientist, started working at XYZ', 'age': 29}': 0.7826511055231093
Combined similarity score with '{'name': 'Alice_9073', 'about': 'Hi, I started working as a Data Scientist at XYZ', 'age': 28}': 0.7275668403255605


#Final test- trying out combined levenshtein and transformer with dataset used to make neo4j network. Added a clone to test results.

In [34]:
import pandas as pd
import Levenshtein
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Function to concatenate profile values into a single string
def concatenate_profile(test):
    return f"{test['name']} {test['city']} {test['position']}"

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Function to compute cosine similarity between a source profile and a list of target profiles
def compute_similarity_scores(source_profile, target_profiles):
    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

    # Concatenate profile values
    source_sentence = concatenate_profile(source_profile)
    target_sentences = [concatenate_profile(profile) for profile in target_profiles]

    # Tokenize sentences
    encoded_input = tokenizer([source_sentence] + target_sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # Calculate cosine similarities
    source_embedding = sentence_embeddings[0]
    target_embeddings = sentence_embeddings[1:]
    cosine_similarities = F.cosine_similarity(source_embedding.unsqueeze(0), target_embeddings, dim=1)

    # Calculate Levenshtein similarities
    levenshtein_similarities = [Levenshtein.ratio(source_sentence, target_sentence) for target_sentence in target_sentences]

    # Combine both similarities
    combined_similarities = [(cos_sim * 0.85 + lev_sim * 0.15) for cos_sim, lev_sim in zip(cosine_similarities.tolist(), levenshtein_similarities)]

    return combined_similarities

# Main code
if __name__ == "__main__":
    # Path to the CSV file containing profiles
    csv_file_path = 'test4.csv'

    # Read profiles from CSV
    df = pd.read_csv(csv_file_path)

    # Source profile (assuming it's the first row in the CSV)
    source_profile = df.iloc[0].to_dict()

    # List of target profiles (excluding the source profile)
    target_profiles = df.iloc[1:].to_dict(orient='records')

    # Compute similarity scores between the source profile and the target profiles
    similarity_scores = compute_similarity_scores(source_profile, target_profiles)

    # Print similarity scores with corresponding profiles
    for score, profile in zip(similarity_scores, target_profiles):
        print(f"Combined similarity score with '{profile['name']}': {score}")
        print(f"Source Profile: {source_profile}")
        print(f"Target Profile: {profile}")
        print()


Combined similarity score with 'margot-bon-51a04624': 0.43076457183763206
Source Profile: {'name': 'catherinemcilkenny', 'city': 'Canada', 'position': 'Snr Business Analyst at Emploi et Développement social Canada (EDSC) / Employment and Social Development Canada (ESDC)'}
Target Profile: {'name': 'margot-bon-51a04624', 'city': 'The Randstad, Netherlands', 'position': 'Communicatieadviseur Corporate & Strategie Gemeente Utrecht'}

Combined similarity score with 'mike-dean-8509a193': 0.2749526411829508
Source Profile: {'name': 'catherinemcilkenny', 'city': 'Canada', 'position': 'Snr Business Analyst at Emploi et Développement social Canada (EDSC) / Employment and Social Development Canada (ESDC)'}
Target Profile: {'name': 'mike-dean-8509a193', 'city': 'England, United Kingdom', 'position': 'Network Data Manager at Network Rail'}

Combined similarity score with 'giovanna-panarella-99a0a4167': 0.12346278778635539
Source Profile: {'name': 'catherinemcilkenny', 'city': 'Canada', 'position': 